# PyCharm ↔ Colab via Jupyter Remote Kernel
Simpler than SSH — just expose Colab's Jupyter server and connect PyCharm to it.
**Requires PyCharm Professional.**

## Step 1: Check GPU

In [ ]:
!nvidia-smi
!python3 --version

## Step 2: Install dependencies & start Jupyter server

In [ ]:
!pip install -q jupyter notebook

# Install cloudflared for tunneling
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb 2>/dev/null
!rm cloudflared-linux-amd64.deb
print("✅ Dependencies installed")

In [ ]:
# Set your token (change this to whatever you want)
TOKEN = "my-colab-token"

# Start Jupyter notebook server in the background
import subprocess

subprocess.Popen([
    "jupyter", "notebook",
    "--ip=0.0.0.0",
    "--port=8888",
    "--no-browser",
    "--allow-root",
    f"--NotebookApp.token={TOKEN}",
    "--NotebookApp.allow_origin=*",
    "--NotebookApp.disable_check_xsrf=True",
], stdout=open("/tmp/jupyter.log", "w"), stderr=subprocess.STDOUT)

import time; time.sleep(3)
print("✅ Jupyter server running on port 8888")
print(f"   Token: {TOKEN}")

## Step 3: Create the tunnel
This exposes the Jupyter server via a public URL. **Copy the URL that appears.**

In [ ]:
import subprocess, re, threading, time

tunnel_url = None

def run_tunnel():
    global tunnel_url
    proc = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://localhost:8888", "--no-autoupdate"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    for line in proc.stdout:
        match = re.search(r'(https://[\w-]+\.trycloudflare\.com)', line)
        if match:
            tunnel_url = match.group(1)
            print("\n" + "="*70)
            print("🚀 JUPYTER TUNNEL READY!")
            print("="*70)
            print(f"\n📋 Use this URL in PyCharm:")
            print(f"\n   {tunnel_url}/?token={TOKEN}")
            print(f"\n   Or separately:")
            print(f"   Server URL:  {tunnel_url}/")
            print(f"   Token:       {TOKEN}")
            print("\n" + "="*70)
            print("\nPyCharm setup:")
            print("  1. Open any .ipynb file in PyCharm")
            print("  2. Click 'Configure Jupyter Server' (top of notebook)")
            print("  3. Select 'Configured Server'")
            print(f"  4. Paste: {tunnel_url}/?token={TOKEN}")
            print("  5. Click OK → run cells on Colab GPU!")
            print("="*70 + "\n")

thread = threading.Thread(target=run_tunnel, daemon=True)
thread.start()

print("⏳ Starting tunnel... (wait ~10 seconds)")
time.sleep(15)
if not tunnel_url:
    print("Still waiting... check output below.")

## Step 4: Keep alive

In [ ]:
import time
from datetime import datetime

print("🔄 Keep-alive running. Leave this cell active.")
print("   Your PyCharm Jupyter connection is live.\n")

counter = 0
while True:
    counter += 1
    if counter % 20 == 0:
        print(f"[{datetime.now().strftime('%H:%M:%S')}] Session alive - {counter * 30}s elapsed")
    time.sleep(30)

## (Optional) Install project packages

In [ ]:
# Uncomment what you need:
# !pip install torch transformers datasets accelerate
# !pip install numpy pandas scikit-learn matplotlib

---
## Troubleshooting

**"Connection refused" in PyCharm:** Re-run the tunnel cell — the URL changes every session.

**Kernel dies mid-execution:** Colab hit a memory limit. Try a higher-tier runtime.

**PyCharm says "untrusted server":** Click through the certificate warning — cloudflare tunnels use self-signed certs.

**Verify Jupyter is running:**

In [ ]:
# Debug: check Jupyter server status
!cat /tmp/jupyter.log | tail -5
!curl -s http://localhost:8888/api/status | python3 -m json.tool